# A bit more of Pandas 
### and some RegEx
<br><br>
<div style="background-color:rgba(128, 128, 0, 0.1); text-align:left; vertical-align: middle; padding:20px 0;">
<p style="font-size:134%;color:Deep Teal;">SC 4125: Developing Data Products</p>
    <p style="font-size:100%;color:Olive;">Module-2: Basics of Pandas structures and manipulation</p><br>

    
<br> 
by <a href="https://personal.ntu.edu.sg/anwitaman/" style="font-size:100%;color:Deep Teal;">Anwitaman DATTA</a><br>
School of Computer Science and Engineering, NTU Singapore.        
</div>

#### Teaching material
- <a href="M2-PanEx.slides.html">.html</a> deck of slides
- <a href="M2-PanEx.ipynb">.ipynb</a> Jupyter notebook

### Disclaimer/Caveat emptor

- Non-systematic and non-exhaustive review
- Example solutions are not necessarily the most efficient or elegant, let alone unique

### Positioning this module in the big picture

<img src="pics/Module-2-In-BigPic.png" alt="Big picture" width="500"/><br>

<img src="pics/cute-panda.png" alt="Basic Pandas Training" width="300"/><br>

- Some basic tools for cleaning and manipulating your data: Apply, Lambdas, Merge, Groupby, Missing data, etc.

In [1]:
# housekeeping - imports necessary libraries
import numpy as np
import pandas as pd
import re
import datetime 

### Load data and carry out some preliminary cleaning

Let's load the same gapminder data we had worked with in the previous module, but from a .tsv file this time. Likewise, let's load the CountriesOfTheWorld data from a .csv file.

In [2]:
# load data
gapminderdatapath ='data/gapminder/' # change this to adjust relative path
gap_df = pd.read_csv(gapminderdatapath+'gapminder.tsv', sep='\t')
gap_df.sample(5)

,country,continent,year,lifeExp,pop,gdpPercap
761,Israel,Asia,1977,73.060,3495918,13306.619210
1263,Reunion,Africa,1967,60.542,414024,4021.175739
787,Jamaica,Americas,1987,71.770,2326606,6351.237495
868,Lebanon,Asia,1972,65.421,2680018,7486.384341
1473,Sweden,Europe,1997,79.390,8897619,25266.594990


In [3]:
# load data
countriesdatapath ='data/'
CoTW_df = pd.read_csv(countriesdatapath+'CountriesOfTheWorld.csv')
CoTW_df.sample(5)

,Country,Region,Population,Area (sq. mi.),Pop. Density (per sq. mi.),Coastline (coast/area ratio),Net migration,Infant mortality (per 1000 births),GDP ($ per capita),Literacy (%),Phones (per 1000),Arable (%),Crops (%),Other (%),Climate,Birthrate,Deathrate,Agriculture,Industry,Service
127,Maldives,ASIA (EX. NEAR EAST),359008,300,"1196,7","214,67",0,"56,52",3900.0,"97,2","90,0","13,33","16,67",70,2,"34,81","7,06","0,2","0,18","0,62"
10,Aruba,LATIN AMER. & CARIB,71891,193,"372,5","35,49",0,"5,89",28000.0,"97,0","516,1","10,53",0,"89,47",2,"11,03","6,68","0,004","0,333","0,663"
130,Marshall Islands,OCEANIA,60422,11854,"5,1","3,12","-6,04","29,45",1600.0,"93,7","91,2","16,67","38,89","44,44",2,"33,05","4,78","0,317","0,149","0,534"
163,Poland,EASTERN EUROPE,38536869,312685,"123,3","0,16","-0,49","8,51",11100.0,"99,8","306,3","45,91","1,12","52,97",3,"9,85","9,89","0,05","0,311","0,64"
185,Slovakia,EASTERN EUROPE,5439448,48845,"111,4","0,00","0,3","7,41",13300.0,NaN,"220,1","30,16","2,62","67,22",3,"10,65","9,45","0,035","0,294","0,672"


What is the list of countries in the gapminder and CoTW datasets?

In [4]:
gap_country_names_series=gap_df['country'].drop_duplicates()
print(gap_country_names_series.to_list()[:10]) #if you print the length, you will get a result of 142

['Afghanistan', 'Albania', 'Algeria', 'Angola', 'Argentina', 'Australia', 'Austria', 'Bahrain', 'Bangladesh', 'Belgium']


In [5]:
CoTW_country_names=CoTW_df['Country'].unique()
print(CoTW_country_names[:10]) #if you print the length, you will get a result of 227

['Afghanistan ' 'Albania ' 'Algeria ' 'American Samoa ' 'Andorra '
 'Angola ' 'Anguilla ' 'Antigua & Barbuda ' 'Argentina ' 'Armenia ']


In [6]:
# Let's get rid of the trailing space in the records
CoTW_df['Country']=CoTW_df.Country.apply(lambda x: x[:-1])
# Alternative: CoTW_df['Country']=CoTW_df.Country.apply(lambda x: x.rstrip()) 
# Alternate way would be to `strip' the space (in this case, rstrip would also work)
CoTW_country_names=CoTW_df['Country'].unique()
print(CoTW_country_names[:10])

['Afghanistan' 'Albania' 'Algeria' 'American Samoa' 'Andorra' 'Angola'
 'Anguilla' 'Antigua & Barbuda' 'Argentina' 'Armenia']


#### Apply & Lambda
- We just saw the use of two very interesting tools (which often go hand-in-hand when processing a data frame):
    1. Apply: Apply a function along an axis of the data frame. It can also be used over a column. 
        * See details at the <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html">doc page</a> 
    2. Lambda: A small anonymous function. You can of-course use a named function inside your lambda, if you need something more complicated.


In [7]:
# load data
olymp_df=pd.read_html(r'https://en.wikipedia.org/wiki/2016_Summer_Olympics_medal_table')
olymp2016medals=olymp_df[2][:-1]
olymp2016medals.sample(5)

,Rank,NOC,Gold,Silver,Bronze,Total
84,78,Trinidad and Tobago,0,0,1,1
15,16,Jamaica,6,3,2,11
35,35,Thailand,2,2,2,6
0,1,United States,46,37,38,121
10,11,Netherlands,8,7,4,19


#### The following code was there because previously, each country name in the table was followed with a (ThreeLetterAcronym)
olymp2016medals['NOC']=olymp2016medals.NOC.apply(lambda x: x.split("(",1)[0][:-1])
olymp2016medals

In [8]:
print(sorted(set(olymp2016medals['NOC'])-set(CoTW_country_names)) )
print(sorted(set(CoTW_country_names)-set(olymp2016medals['NOC'])))
# Note that several of the same countries are named or spelled differently in the two lists

['Bahamas', 'Brazil*', 'Chinese Taipei', 'Great Britain', 'Independent Olympic Athletes', 'Ivory Coast', 'Kosovo', 'North Korea', 'South Korea', 'Trinidad and Tobago']
['Afghanistan', 'Albania', 'American Samoa', 'Andorra', 'Angola', 'Anguilla', 'Antigua & Barbuda', 'Aruba', 'Bahamas, The', 'Bangladesh', 'Barbados', 'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Bolivia', 'Bosnia & Herzegovina', 'Botswana', 'Brazil', 'British Virgin Is.', 'Brunei', 'Burkina Faso', 'Burma', 'Cambodia', 'Cameroon', 'Cape Verde', 'Cayman Islands', 'Central African Rep.', 'Chad', 'Chile', 'Comoros', 'Congo, Dem. Rep.', 'Congo, Repub. of the', 'Cook Islands', 'Costa Rica', "Cote d'Ivoire", 'Cyprus', 'Djibouti', 'Dominica', 'East Timor', 'Ecuador', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Faroe Islands', 'French Guiana', 'French Polynesia', 'Gabon', 'Gambia, The', 'Gaza Strip', 'Ghana', 'Gibraltar', 'Greenland', 'Guadeloupe', 'Guam', 'Guatemala', 'Guernsey', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Haiti', 'Ho

In [9]:
CoTW_df.loc[(CoTW_df.Country=="Bahamas, The"),'Country']='Bahamas'
CoTW_df.loc[(CoTW_df.Country=='Taiwan'),'Country']='Chinese Taipei'
CoTW_df.loc[(CoTW_df.Country=='United Kingdom'),'Country']='Great Britain'
CoTW_df.loc[(CoTW_df.Country=="Cote d'Ivoire"),'Country']='Ivory Coast'
CoTW_df.loc[(CoTW_df.Country=='Korea, North'),'Country']='North Korea'
CoTW_df.loc[(CoTW_df.Country=='Korea, South'),'Country']='South Korea'
CoTW_df.loc[(CoTW_df.Country=="Trinidad & Tobago"),'Country']='Trinidad and Tobago'
olymp2016medals.loc[(olymp2016medals.NOC=="Brazil*"),'NOC']='Brazil'
CoTW_country_names=CoTW_df['Country'].unique()
print(sorted(set(olymp2016medals['NOC'])-set(CoTW_country_names)) )
#print(sorted(set(CoTW_country_names)-set(olymp2016medals['NOC'])))

['Independent Olympic Athletes', 'Kosovo']


C:\Users\datta\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


Need to `fix' all the anomalous data, particularly if we want to correlate the information across the data frames. Above, I did this manually.

Talking points: 
1. Challenges to do this at scale.
    * Check for similarity of words: e.g., Korea, North vs North Korea; Trindad & Tobago vs Trinidad and Tobago
    * What about Cote d'Ivoire vs Ivory Coast? Swaziland versus Switzerland? Niger vs Nigeria?
    * What to do with United Kingdom vs Great Britain, Taiwan vs Chinese Taipei?
2. Sometimes, you may be lucky and can leverage on Named entity recognition and matching tools.

Caveat: Different domains may be more or less amenable to certain solutions/heuristics. 

#### Detour

For the particular problem at hand, namely, each country has been spelled variously, and we want to identify them to be equivalent, instead of writing our own tool, we may leverage on a nice tool someone already built!

Namely, country_converter: https://github.com/konstantinstadler/country_converter

There's also pycountry, which is useful in some related cases, but it maynot be suitable for our particular scenario (at least, I didn't figure out how to use it for our problem): https://pypi.org/project/pycountry/ 

In [10]:
#!pip install country_converter --upgrade
import country_converter as coco

In [11]:
# sanity check
some_names = ['Niger', 'Nigeria', 'Korea, North', "North Korea", "Cote d'Ivoire", "Ivory Coast", "Brazil*", "Brazil", "Great Britain", "United Kingdom", "Taiwan", "Chinese taipei", "Trinidad & Tobago", "Trinidad and Tobago", "Alice's Wonderland"]
standard_names = coco.convert(names=some_names, to='name_short')
print(standard_names)

Alice's Wonderland not found in regex


['Niger', 'Nigeria', 'North Korea', 'North Korea', "Cote d'Ivoire", "Cote d'Ivoire", 'Brazil', 'Brazil', 'United Kingdom', 'United Kingdom', 'Taiwan', 'Taiwan', 'Trinidad and Tobago', 'Trinidad and Tobago', 'not found']


#### Merge data frames
- how: {‘left’, ‘right’, ‘outer’, ‘inner’, ‘cross’}, default ‘inner’
    * <a href="https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html">Pandas doc</a>
    * A nice discussion on various options how merger is done can be at <a href="https://stackoverflow.com/questions/448023/what-is-the-difference-between-left-right-outer-and-inner-joins">Stack Overflow</a>. [Below figure is from there.]
    
<img src="pics/joins-overflow.png" alt="kinds of joins" width="350">    

- Note that Pandas has also a Join method, which however in-fact join columns with other DataFrame either on index or on a key column. <a href="https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html">A good resource to check</a>.

In [12]:
# Assuming that the data has been `cleaned' 
# Determine all the countries with a population of more than 20 million 
# as per the CountriesOfTheWorld table, that got no gold medals.  
Merged_df=olymp2016medals.merge(CoTW_df[['Country','Population']],left_on='NOC', right_on='Country')
Merged_df[(Merged_df['Population']>20000000) & (Merged_df['Gold']==0)]
# Note: For the specific purpose, from CoTW_df, I do not need the other columns than Country and Population
# I could have cleaned the data more, e.g. NOC and Country columns are redundant.

,Rank,NOC,Gold,Silver,Bronze,Total,Country,Population
57,60,Malaysia,0,4,1,5,Malaysia,24385858
58,61,Mexico,0,3,2,5,Mexico,107449525
59,62,Venezuela,0,2,1,3,Venezuela,25730435
60,63,Algeria,0,2,0,2,Algeria,32930091
64,67,India,0,1,1,2,India,1095351995
69,69,Philippines,0,1,0,1,Philippines,89468677
72,75,Egypt,0,0,3,3,Egypt,78887007
79,78,Morocco,0,0,1,1,Morocco,33241259
80,78,Nigeria,0,0,1,1,Nigeria,131859731


<b>Ungraded task 2.1: Ratio of total medals to per capita GDP</b><br><br>

Determine the ratio of the number of total medals to the per capita GDP for each country as per the CountriesOfTheWorld table and sort them in decreasing order. Compare the results with others.

- Discuss: What subtleties are involved? 
    * How do you account for the countries which are not in the original Olympics table?

#### Groupby, Hierarchical index

In [13]:
# For each year for which data is available in the gap_df Dataframe
# What was the minimum recorded life expectancy?
gap_df.groupby(["year"])["lifeExp"].min()                                                    

year
1952    28.801
1957    30.332
1962    31.997
1967    34.020
1972    35.400
1977    31.220
1982    38.445
1987    39.906
1992    23.599
1997    36.087
2002    39.193
2007    39.613
Name: lifeExp, dtype: float64

In [14]:
# What if we also want to know, which country had said minimum life expectancy? 
gap_df.loc[gap_df.groupby(["year"])["lifeExp"].idxmin()][['year','country','lifeExp']]                                                        

,year,country,lifeExp
0,1952,Afghanistan,28.801
1,1957,Afghanistan,30.332
2,1962,Afghanistan,31.997
3,1967,Afghanistan,34.020
1348,1972,Sierra Leone,35.400
221,1977,Cambodia,31.220
1350,1982,Sierra Leone,38.445
43,1987,Angola,39.906
1292,1992,Rwanda,23.599
1293,1997,Rwanda,36.087


In [15]:
# Groupby can be useful in creating hierarchical indexing
# e.g., When we want to check the data by regions.
CoTW_df.groupby(['Region','Country']).mean()
# Many of the data, though representing numeric values, are in-fact treated as string in the current Dataframe. 
# Refer to Ungraded Task 2.2 (below) for more on this. The nature of the data may limit what can be computed.  

Population  Area (sq. mi.)  \
Region                              Country                                   
ASIA (EX. NEAR EAST)                Afghanistan    31056997          647500   
                                    Bangladesh    147365352          144000   
                                    Bhutan          2279723           47000   
                                    Brunei           379444            5770   
                                    Burma          47382633          678500   
...                                                     ...             ...   
WESTERN EUROPE                      Portugal       10605870           92391   
                                    San Marino        29251              61   
                                    Spain          40397842          504782   
                                    Sweden          9016596          449964   
                                    Switzerland     7523934           41290   

                                                 GDP ($ per capita)  
Region                              Country                          
ASIA (EX. NEAR EAST)                Afghanistan               700.0  
                                    Bangladesh               1900.0  
                                    Bhutan                   1300.0  
                                    Brunei                  18600.0  
                                    Burma                    1800.0  
...                                                             ...  
WESTERN EUROPE                      Portugal                18000.0  
                                    San Marino              34600.0  
                                    Spain                   22000.0  
                                    Sweden                  26800.0  
                                    Switzerland             32700.0  

[227 rows x 3 columns]

In [16]:
CoTW_df.groupby(['Region','Country']).max()
# With something like max, Pandas seems more generous in ignoring data type ;)

Population  Area (sq. mi.)  \
Region                              Country                                   
ASIA (EX. NEAR EAST)                Afghanistan    31056997          647500   
                                    Bangladesh    147365352          144000   
                                    Bhutan          2279723           47000   
                                    Brunei           379444            5770   
                                    Burma          47382633          678500   
...                                                     ...             ...   
WESTERN EUROPE                      Portugal       10605870           92391   
                                    San Marino        29251              61   
                                    Spain          40397842          504782   
                                    Sweden          9016596          449964   
                                    Switzerland     7523934           41290   

                                                Pop. Density (per sq. mi.)  \
Region                              Country                                  
ASIA (EX. NEAR EAST)                Afghanistan                       48,0   
                                    Bangladesh                      1023,4   
                                    Bhutan                            48,5   
                                    Brunei                            65,8   
                                    Burma                             69,8   
...                                                                    ...   
WESTERN EUROPE                      Portugal                         114,8   
                                    San Marino                       479,5   
                                    Spain                             80,0   
                                    Sweden                            20,0   
                                    Switzerland                      182,2   

                                                Coastline (coast/area ratio)  \
Region                              Country                                    
ASIA (EX. NEAR EAST)                Afghanistan                         0,00   
                                    Bangladesh                          0,40   
                                    Bhutan                              0,00   
                                    Brunei                              2,79   
                                    Burma                               0,28   
...                                                                      ...   
WESTERN EUROPE                      Portugal                            1,94   
                                    San Marino                          0,00   
                                    Spain                               0,98   
                                    Sweden                              0,72   
                                    Switzerland                         0,00   

                                                Net migration  \
Region                              Country                     
ASIA (EX. NEAR EAST)                Afghanistan         23,06   
                                    Bangladesh          -0,71   
                                    Bhutan                  0   
                                    Brunei               3,59   
                                    Burma                -1,8   
...                                                       ...   
WESTERN EUROPE                      Portugal             3,57   
                                    San Marino          10,98   
                                    Spain                0,99   
                                    Sweden               1,67   
                                    Switzerland          4,05   

                                                Infant mortality (per 1000 births)  \
Region                           

In [17]:
# For what we want, an alternate, and simpler (and more robust, since we don't need to worry about data types) 
# option is to just use create hierarchical index using set_index, and then sort_index.
CoTW_df.set_index(['Region','Country']).sort_index()

Population  Area (sq. mi.)  \
Region                              Country                                   
ASIA (EX. NEAR EAST)                Afghanistan    31056997          647500   
                                    Bangladesh    147365352          144000   
                                    Bhutan          2279723           47000   
                                    Brunei           379444            5770   
                                    Burma          47382633          678500   
...                                                     ...             ...   
WESTERN EUROPE                      Portugal       10605870           92391   
                                    San Marino        29251              61   
                                    Spain          40397842          504782   
                                    Sweden          9016596          449964   
                                    Switzerland     7523934           41290   

                                                Pop. Density (per sq. mi.)  \
Region                              Country                                  
ASIA (EX. NEAR EAST)                Afghanistan                       48,0   
                                    Bangladesh                      1023,4   
                                    Bhutan                            48,5   
                                    Brunei                            65,8   
                                    Burma                             69,8   
...                                                                    ...   
WESTERN EUROPE                      Portugal                         114,8   
                                    San Marino                       479,5   
                                    Spain                             80,0   
                                    Sweden                            20,0   
                                    Switzerland                      182,2   

                                                Coastline (coast/area ratio)  \
Region                              Country                                    
ASIA (EX. NEAR EAST)                Afghanistan                         0,00   
                                    Bangladesh                          0,40   
                                    Bhutan                              0,00   
                                    Brunei                              2,79   
                                    Burma                               0,28   
...                                                                      ...   
WESTERN EUROPE                      Portugal                            1,94   
                                    San Marino                          0,00   
                                    Spain                               0,98   
                                    Sweden                              0,72   
                                    Switzerland                         0,00   

                                                Net migration  \
Region                              Country                     
ASIA (EX. NEAR EAST)                Afghanistan         23,06   
                                    Bangladesh          -0,71   
                                    Bhutan                  0   
                                    Brunei               3,59   
                                    Burma                -1,8   
...                                                       ...   
WESTERN EUROPE                      Portugal             3,57   
                                    San Marino          10,98   
                                    Spain                0,99   
                                    Sweden               1,67   
                                    Switzerland          4,05   

                                                Infant mortality (per 1000 births)  \
Region                           

#### Missing data 

There are many reasons that data may be missing. e.g.:
- It may be inherently missing: not collected, not entered, etc.
- It may be implicitly missing.
- Missing entries may get created during data processing. 

In [18]:
# load data (source: https://github.com/chendaniely/pandas_for_everyone) 
pfe_path ='data/pandas_for_everyone_data/' # change this to adjust relative path
#visited_df = pd.read_csv(pfe_path+'survey_visited.csv')
#survey_df = pd.read_csv(pfe_path+'survey_survey.csv')
ebola_df= pd.read_csv(pfe_path+'country_timeseries.csv')
ebola_df.sample(6)

,Date,Day,Cases_Guinea,Cases_Liberia,Cases_SierraLeone,Cases_Nigeria,Cases_Senegal,Cases_UnitedStates,Cases_Spain,Cases_Mali,Deaths_Guinea,Deaths_Liberia,Deaths_SierraLeone,Deaths_Nigeria,Deaths_Senegal,Deaths_UnitedStates,Deaths_Spain,Deaths_Mali
108,4/14/2014,23,168.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,108.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88,6/3/2014,73,344.0,13.0,NaN,NaN,NaN,NaN,NaN,NaN,215.0,12.0,6.0,NaN,NaN,NaN,NaN,NaN
100,4/24/2014,33,NaN,35.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
35,10/19/2014,211,1540.0,NaN,3706.0,20.0,1.0,3.0,1.0,NaN,904.0,NaN,1259.0,8.0,0.0,1.0,0.0,NaN
22,11/11/2014,234,1919.0,NaN,5586.0,20.0,1.0,4.0,1.0,4.0,1166.0,NaN,1187.0,8.0,0.0,1.0,0.0,3.0
96,5/5/2014,44,235.0,13.0,0.0,NaN,NaN,NaN,NaN,NaN,157.0,11.0,0.0,NaN,NaN,NaN,NaN,NaN


In [19]:
# One option is to consider missing data to have some default value.
# e.g., in the case of number of Ebola cases, it may be fair to consider missing values to be 0.
ebola_df.fillna(0,inplace=True)
ebola_df

,Date,Day,Cases_Guinea,Cases_Liberia,Cases_SierraLeone,Cases_Nigeria,Cases_Senegal,Cases_UnitedStates,Cases_Spain,Cases_Mali,Deaths_Guinea,Deaths_Liberia,Deaths_SierraLeone,Deaths_Nigeria,Deaths_Senegal,Deaths_UnitedStates,Deaths_Spain,Deaths_Mali
0,1/5/2015,289,2776.0,0.0,10030.0,0.0,0.0,0.0,0.0,0.0,1786.0,0.0,2977.0,0.0,0.0,0.0,0.0,0.0
1,1/4/2015,288,2775.0,0.0,9780.0,0.0,0.0,0.0,0.0,0.0,1781.0,0.0,2943.0,0.0,0.0,0.0,0.0,0.0
2,1/3/2015,287,2769.0,8166.0,9722.0,0.0,0.0,0.0,0.0,0.0,1767.0,3496.0,2915.0,0.0,0.0,0.0,0.0,0.0
3,1/2/2015,286,0.0,8157.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3496.0,0.0,0.0,0.0,0.0,0.0,0.0
4,12/31/2014,284,2730.0,8115.0,9633.0,0.0,0.0,0.0,0.0,0.0,1739.0,3471.0,2827.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,3/27/2014,5,103.0,8.0,6.0,0.0,0.0,0.0,0.0,0.0,66.0,6.0,5.0,0.0,0.0,0.0,0.0,0.0
118,3/26/2014,4,86.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,62.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
119,3/25/2014,3,86.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
120,3/24/2014,2,86.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,59.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
# Let's consider an example of how missing data may get created during the processing (because it was implicitly missing)
afg_df = gap_df[gap_df['country']=='Afghanistan'][['year','pop','gdpPercap']].tail(2).set_index('year')
afg_df

,pop,gdpPercap
year,,
2002,25268405,726.734055
2007,31889923,974.580338


In [21]:
# If we for some reason need yearly population and per capita GDP
# That information was missing implicitly, and during our processing, it may get so explicitly
afg_df=afg_df.reindex(afg_df.index.tolist() + list(range(2000, 2011)))
afg_df=afg_df[~afg_df.index.duplicated(keep='first')].sort_index()
afg_df

,pop,gdpPercap
year,,
2000,NaN,NaN
2001,NaN,NaN
2002,25268405.0,726.734055
2003,NaN,NaN
2004,NaN,NaN
2005,NaN,NaN
2006,NaN,NaN
2007,31889923.0,974.580338
2008,NaN,NaN


In [22]:
# forward fill (the population)
afg_df.loc[:,'pop'] = afg_df.loc[:,'pop'].ffill()
afg_df

,pop,gdpPercap
year,,
2000,NaN,NaN
2001,NaN,NaN
2002,25268405.0,726.734055
2003,25268405.0,NaN
2004,25268405.0,NaN
2005,25268405.0,NaN
2006,25268405.0,NaN
2007,31889923.0,974.580338
2008,31889923.0,NaN


In [23]:
# backward fill (the population)
afg_df.loc[:,'pop'] = afg_df.loc[:,'pop'].bfill()
afg_df

,pop,gdpPercap
year,,
2000,25268405.0,NaN
2001,25268405.0,NaN
2002,25268405.0,726.734055
2003,25268405.0,NaN
2004,25268405.0,NaN
2005,25268405.0,NaN
2006,25268405.0,NaN
2007,31889923.0,974.580338
2008,31889923.0,NaN


In [24]:
# interpolate fill: many caveats apply
# only linear interpolation is supported
# the gaps are filled with linear interpolation. 
# the extremities just repeat the data

afg_df.loc[:,'gdpPercap'] = afg_df.loc[:,'gdpPercap'].interpolate(limit_direction='both')
afg_df
# you may need your own custom method

,pop,gdpPercap
year,,
2000,25268405.0,726.734055
2001,25268405.0,726.734055
2002,25268405.0,726.734055
2003,25268405.0,776.303312
2004,25268405.0,825.872568
2005,25268405.0,875.441825
2006,25268405.0,925.011082
2007,31889923.0,974.580338
2008,31889923.0,974.580338


In [25]:
# Be mindful regarding the semantics of the data
# This data is from a 'toy example' on video playbacks 
# from Prof. Christopher Brooks' course on Data Science in Python 
# Usage logs being collected from multiple users parallely, with some missing data
data_from_coursera_path ='data/coursera_sourced_data/'
log_df=pd.read_csv(data_from_coursera_path+'log.csv')
log_df

,time,user,video,playback position,paused,volume
0,1469974424,cheryl,intro.html,5,False,10.0
1,1469974454,cheryl,intro.html,6,NaN,NaN
2,1469974544,cheryl,intro.html,9,NaN,NaN
3,1469974574,cheryl,intro.html,10,NaN,NaN
4,1469977514,bob,intro.html,1,NaN,NaN
5,1469977544,bob,intro.html,1,NaN,NaN
6,1469977574,bob,intro.html,1,NaN,NaN
7,1469977604,bob,intro.html,1,NaN,NaN
8,1469974604,cheryl,intro.html,11,NaN,NaN
9,1469974694,cheryl,intro.html,14,NaN,NaN


In [26]:
# Let's view the data sorted by time
log_df=log_df.set_index('time').sort_index() 
log_df

,user,video,playback position,paused,volume
time,,,,,
1469974424,cheryl,intro.html,5,False,10.0
1469974424,sue,advanced.html,23,False,10.0
1469974454,cheryl,intro.html,6,NaN,NaN
1469974454,sue,advanced.html,24,NaN,NaN
1469974484,cheryl,intro.html,7,NaN,NaN
1469974514,cheryl,intro.html,8,NaN,NaN
1469974524,sue,advanced.html,25,NaN,NaN
1469974544,cheryl,intro.html,9,NaN,NaN
1469974554,sue,advanced.html,26,NaN,NaN


In [27]:
log_df=log_df.reset_index().set_index(['user','time']).sort_index()
# We want to use a 2-level index, so that all entries for a given user are together
# Chaining can keep your code short. Make sure it does not make it too cryptic though ;) 
# Motivation: Using user Cheryl's volume setting to fill-in user Bob's volume may not be meaningful
log_df

video  playback position paused  volume
user   time                                                       
bob    1469977424     intro.html                  1   True    10.0
       1469977454     intro.html                  1    NaN     NaN
       1469977484     intro.html                  1    NaN     NaN
       1469977514     intro.html                  1    NaN     NaN
       1469977544     intro.html                  1    NaN     NaN
       1469977574     intro.html                  1    NaN     NaN
       1469977604     intro.html                  1    NaN     NaN
       1469977634     intro.html                  1    NaN     NaN
       1469977664     intro.html                  1    NaN     NaN
       1469977694     intro.html                  1    NaN     NaN
       1469977724     intro.html                  1    NaN     NaN
cheryl 1469974424     intro.html                  5  False    10.0
       1469974454     intro.html                  6    NaN     NaN
       1469974484     intro.html                  7    NaN     NaN
       1469974514     intro.html                  8    NaN     NaN
       1469974544     intro.html                  9    NaN     NaN
       1469974574     intro.html                 10    NaN     NaN
       1469974604     intro.html                 11    NaN     NaN
       1469974634     intro.html                 12    NaN     NaN
       1469974664     intro.html                 13    NaN     NaN
       1469974694     intro.html                 14    NaN     NaN
       1469974724     intro.html                 15    NaN     NaN
sue    1469974424  advanced.html                 23  False    10.0
       1469974454  advanced.html                 24    NaN     NaN
       1469974524  advanced.html                 25    NaN     NaN
       1469974554  advanced.html                 26    NaN     NaN
       1469974624  advanced.html                 27    NaN     NaN
       1469974654  advanced.html                 28    NaN     5.0
       1469974724  advanced.html                 29    NaN     NaN
       1469974754  advanced.html                 30    NaN     NaN
       1469974824  advanced.html                 31    NaN     NaN
       1469974854  advanced.html                 32    NaN     NaN
       1469974924  advanced.html                 33    NaN     NaN

In [28]:
# Now, it makes certain sense to assume that 
# the missing values correspond to the last recorded value for that particular user
# Forward the Fill! (based on a reasonable Foundation of the assumptions ;)
log_df = log_df.fillna(method='ffill')
log_df

video  playback position  paused  volume
user   time                                                        
bob    1469977424     intro.html                  1    True    10.0
       1469977454     intro.html                  1    True    10.0
       1469977484     intro.html                  1    True    10.0
       1469977514     intro.html                  1    True    10.0
       1469977544     intro.html                  1    True    10.0
       1469977574     intro.html                  1    True    10.0
       1469977604     intro.html                  1    True    10.0
       1469977634     intro.html                  1    True    10.0
       1469977664     intro.html                  1    True    10.0
       1469977694     intro.html                  1    True    10.0
       1469977724     intro.html                  1    True    10.0
cheryl 1469974424     intro.html                  5   False    10.0
       1469974454     intro.html                  6   False    10.0
       1469974484     intro.html                  7   False    10.0
       1469974514     intro.html                  8   False    10.0
       1469974544     intro.html                  9   False    10.0
       1469974574     intro.html                 10   False    10.0
       1469974604     intro.html                 11   False    10.0
       1469974634     intro.html                 12   False    10.0
       1469974664     intro.html                 13   False    10.0
       1469974694     intro.html                 14   False    10.0
       1469974724     intro.html                 15   False    10.0
sue    1469974424  advanced.html                 23   False    10.0
       1469974454  advanced.html                 24   False    10.0
       1469974524  advanced.html                 25   False    10.0
       1469974554  advanced.html                 26   False    10.0
       1469974624  advanced.html                 27   False    10.0
       1469974654  advanced.html                 28   False     5.0
       1469974724  advanced.html                 29   False     5.0
       1469974754  advanced.html                 30   False     5.0
       1469974824  advanced.html                 31   False     5.0
       1469974854  advanced.html                 32   False     5.0
       1469974924  advanced.html                 33   False     5.0

## Some basic text processing & RegEx
<a href="https://www.explainxkcd.com/wiki/index.php/1638:_Backslashes"><img src="pics/backslashes.png" alt="XKCD backslashes" width="400"/></a><br>



In [29]:
# Tokens
NTU_tweet ="""Doctors may soon identify #diabetes patients who have a higher risk of vascular #inflammation with a one-step process using a ‘lab-on-a-chip’ device developed by scientists from @NTUsg, @TTSH and @MIT. #NTUsg2025 #NTUsgResearch https://bit.ly/ExoDFFchip"""
Tweet_tokens=NTU_tweet.split()
print(Tweet_tokens)

['Doctors', 'may', 'soon', 'identify', '#diabetes', 'patients', 'who', 'have', 'a', 'higher', 'risk', 'of', 'vascular', '#inflammation', 'with', 'a', 'one-step', 'process', 'using', 'a', '‘lab-on-a-chip’', 'device', 'developed', 'by', 'scientists', 'from', '@NTUsg,', '@TTSH', 'and', '@MIT.', '#NTUsg2025', '#NTUsgResearch', 'https://bit.ly/ExoDFFchip']


In [30]:
# Find 'long' words (more than 4 letters) which are not hashtags, call outs, or urls!
Tweet_longwords=[x for x in Tweet_tokens if (len(x)>4) & (x.find('#')<0) & (x.find('@')<0) & (x.find('://')<0)]
print(Tweet_longwords)

['Doctors', 'identify', 'patients', 'higher', 'vascular', 'one-step', 'process', 'using', '‘lab-on-a-chip’', 'device', 'developed', 'scientists']


In [31]:
# What about, if we want to find all the hashtags?

# Attempt 1:
Tweet_hashtags1a=[x for x in Tweet_tokens if (x.find('#')>=0) ]
Tweet_tokens.append("stranger#things")
Tweet_hashtags1b=[x for x in Tweet_tokens if (x.find('#')>=0) ]

print(Tweet_hashtags1a)
print(Tweet_hashtags1b)

['#diabetes', '#inflammation', '#NTUsg2025', '#NTUsgResearch']
['#diabetes', '#inflammation', '#NTUsg2025', '#NTUsgResearch', 'stranger#things']


In [32]:
# Attempt 2:
# Just in case there's something like `stranger#things' which is not a hashtag!
# We might want to use one of the following:
Tweet_hashtags2=[x for x in Tweet_tokens if (x.find('#')==0) ]  
Tweet_hashtags3=[x for x in Tweet_tokens if x.startswith('#') ]

print(Tweet_hashtags2)
print(Tweet_hashtags3)

['#diabetes', '#inflammation', '#NTUsg2025', '#NTUsgResearch']
['#diabetes', '#inflammation', '#NTUsg2025', '#NTUsgResearch']


In [33]:
# Find all mentions of 'NTU'
Contains_NTU1=[x for x in Tweet_tokens if (x.find('NTU')>=0) ]
Contains_NTU2=[x for x in Tweet_tokens if 'NTU' in x]

print(Contains_NTU1)
print(Contains_NTU2)

['@NTUsg,', '#NTUsg2025', '#NTUsgResearch']
['@NTUsg,', '#NTUsg2025', '#NTUsgResearch']


Some frequently used & useful string methods 

- checking: isalpha/isdigit/isalnum 
- transforming: lower/upper/title/replace/strip/rstrip
- locate: find/rfind

See [https://docs.python.org/3/library/stdtypes.html] for a more exhaustive list.

In [34]:
# Let's redo the extraction of hashtags with a RegEx

[x for x in Tweet_tokens if re.search('\B#[A-Za-z-0-9_]+',x)] 

# For the specifc example tweet, if we used re.search('#',x), it would have sufficed.
# But then it would not have managed to handle the 'stranger' cases such as isolated #, ## or hash sandwiched by text..

['#diabetes', '#inflammation', '#NTUsg2025', '#NTUsgResearch']

#### How to interpret <span style="color:red">'\B#[A-Za-z-0-9_]+'</span>?
     [x for x in Tweet_tokens if re.search('\B#[A-Za-z-0-9_]+',x)] 
     
- <span style="color:red">\B</span> Matches boundary with non-alphanumeric characters

- <span style="color:red">[A-Za-z-0-9_]</span> Matches any of A to Z, a to z, 0 to 9 and _

- <span style="color:red">+</span> Greedily matches that expression on its immediate left is present at least once.

In [35]:
# The same RegEx can be rewritten in a much shorter (and slightly cryptic) manner
[x for x in Tweet_tokens if re.search('\B#\w+',x)]

['#diabetes', '#inflammation', '#NTUsg2025', '#NTUsgResearch']

#### RegEx cheatsheet snippet
Source: <a href="https://www.dataquest.io/wp-content/uploads/2019/03/python-regular-expressions-cheat-sheet.pdf">Dataquest.io</a>

<img src="pics/regex-charclass.png" alt="RegExCharacter Class" width="300"/>

#### RegEx cheatsheet snippet
Source: <a href="https://www.dataquest.io/wp-content/uploads/2019/03/python-regular-expressions-cheat-sheet.pdf">Dataquest.io</a>

<img src="pics/regex-specialchar.png" alt="RegExSpecialChar" width="240"/>



In [36]:
# Open a file, read it line-by-line and create a Data Series 
# The following data is obtained from Prof V. Vydiswaran's course on Applied Text Mining in Python

doc = []
with open(data_from_coursera_path+'text-with-dates.txt','r') as file:
    for line in file:
        doc.append(line.rstrip()) # used rstrip to remove trailing newlines \n

datetxt_df = pd.Series(doc)
datetxt_df

0             03/25/93 Total time of visit (in minutes):
1                           6/18/85 Primary Care Doctor:
2      sshe plans to move as of 7/8/71 In-Home Servic...
3                    7 on 9/27/75 Audit C Score Current:
4      2/6/96 sleep studyPain Treatment Pain Level (N...
                             ...                        
495    1979 Family Psych History: Family History of S...
496    therapist and friend died in ~2006 Parental/Ca...
497                           2008 partial thyroidectomy
498    sPt describes a history of sexual abuse as a c...
499    . In 1980, patient was living in Naples and de...
Length: 500, dtype: object

In [37]:
# What does the following code do?
[x for x in datetxt_df if re.findall(r'\d{1,2}[/|-]\d{1,2}[/|-]\d{2,4}',x)]

['03/25/93 Total time of visit (in minutes):',
 '6/18/85 Primary Care Doctor:',
 'sshe plans to move as of 7/8/71 In-Home Services: None',
 '7 on 9/27/75 Audit C Score Current:',
 '2/6/96 sleep studyPain Treatment Pain Level (Numeric Scale): 7',
 '.Per 7/06/79 Movement D/O note:',
 "4, 5/18/78 Patient's thoughts about current substance abuse:",
 '10/24/89 CPT Code: 90801 - Psychiatric Diagnosis Interview',
 '3/7/86 SOS-10 Total Score:',
 '(4/10/71)Score-1Audit C Score Current:',
 '(5/11/85) Crt-1.96, BUN-26; AST/ALT-16/22; WBC_12.6Activities of Daily Living (ADL) Bathing: Independent',
 '4/09/75 SOS-10 Total Score:',
 '8/01/98 Communication with referring physician?: Done',
 '1/26/72 Communication with referring physician?: Not Done',
 '5/24/1990 CPT Code: 90792: With medical services',
 '1/25/2011 CPT Code: 90792: With medical services',
 '4/12/82 Total time of visit (in minutes):',
 '1; 10/13/1976 Audit C Score, Highest/Date:',
 '4, 4/24/98 Relevant Drug History:',
 ') 59 yo unemploy

In [38]:
[x for x in datetxt_df if re.search(r'\d{1,2}[/|-]\d{1,2}[/|-]\d{2,4}',x)]

# Search returns the first occurrence, while findall finds all the (non-overlapping) matches.
# For our usage here, either suffices.
# Even re.match() would suffice for the current purpose. 
# Check the subtle differences between match/search/findall for yourself! 

['03/25/93 Total time of visit (in minutes):',
 '6/18/85 Primary Care Doctor:',
 'sshe plans to move as of 7/8/71 In-Home Services: None',
 '7 on 9/27/75 Audit C Score Current:',
 '2/6/96 sleep studyPain Treatment Pain Level (Numeric Scale): 7',
 '.Per 7/06/79 Movement D/O note:',
 "4, 5/18/78 Patient's thoughts about current substance abuse:",
 '10/24/89 CPT Code: 90801 - Psychiatric Diagnosis Interview',
 '3/7/86 SOS-10 Total Score:',
 '(4/10/71)Score-1Audit C Score Current:',
 '(5/11/85) Crt-1.96, BUN-26; AST/ALT-16/22; WBC_12.6Activities of Daily Living (ADL) Bathing: Independent',
 '4/09/75 SOS-10 Total Score:',
 '8/01/98 Communication with referring physician?: Done',
 '1/26/72 Communication with referring physician?: Not Done',
 '5/24/1990 CPT Code: 90792: With medical services',
 '1/25/2011 CPT Code: 90792: With medical services',
 '4/12/82 Total time of visit (in minutes):',
 '1; 10/13/1976 Audit C Score, Highest/Date:',
 '4, 4/24/98 Relevant Drug History:',
 ') 59 yo unemploy

#### pandas.Series.str.extract & extractall

Extract captures groups in the regex pattern as columns in DataFrame.

In [39]:
# Extract: Grouping done using ()
datetxt_df.str.extract(r'(\d{1,2}[/|-]\d{1,2}[/|-]\d{2,4})')

,0
0,03/25/93
1,6/18/85
2,7/8/71
3,9/27/75
4,2/6/96
...,...
495,NaN
496,NaN
497,NaN
498,NaN


In [40]:
# Extract all
datetxt_df.str.extractall(r'(\d{1,2}[/|-]\d{1,2}[/|-]\d{2,4})')

,,0
,match,
0,0,03/25/93
1,0,6/18/85
2,0,7/8/71
3,0,9/27/75
4,0,2/6/96
...,...,...
120,0,12/22/98
121,0,10/02/96
122,0,11/05/90


In [41]:
# Further grouping with () 
datetxt_df.str.extractall(r'(\d{1,2})[/|-](\d{1,2})[/|-](\d{2,4})')

,,0,1,2
,match,,,
0,0,03,25,93
1,0,6,18,85
2,0,7,8,71
3,0,9,27,75
4,0,2,6,96
...,...,...,...,...
120,0,12,22,98
121,0,10,02,96
122,0,11,05,90


In [42]:
# Grouping can be carried out in a nested fashion!
datetxt_df.str.extractall(r'((\d{1,2})[/|-]((\d{1,2})[/|-](\d{2,4})))')

,,0,1,2,3,4
,match,,,,,
0,0,03/25/93,03,25/93,25,93
1,0,6/18/85,6,18/85,18,85
2,0,7/8/71,7,8/71,8,71
3,0,9/27/75,9,27/75,27,75
4,0,2/6/96,2,6/96,6,96
...,...,...,...,...,...,...
120,0,12/22/98,12,22/98,22,98
121,0,10/02/96,10,02/96,02,96
122,0,11/05/90,11,05/90,05,90


In [43]:
# Named groups ?P<NamedGroup> 
datetxt_df.str.extract(r'(?P<Date>(?P<Month>\d{1,2})[/|-](?P<Day>\d{1,2})[/|-](?P<Year>\d{2,4}))')

,Date,Month,Day,Year
0,03/25/93,03,25,93
1,6/18/85,6,18,85
2,7/8/71,7,8,71
3,9/27/75,9,27,75
4,2/6/96,2,6,96
...,...,...,...,...
495,NaN,NaN,NaN,NaN
496,NaN,NaN,NaN,NaN
497,NaN,NaN,NaN,NaN
498,NaN,NaN,NaN,NaN


In [44]:
# It can come handy to add derived columns, e.g. with a .join method to join Dataframes
# Since datetxt_df is a Pandas Series, we first need to convert it into a Dataframe
datetxt_df.to_frame(name="Original_txt")\
.join(datetxt_df.str.extract\
(r'(?P<Date>(?P<Month>\d{1,2})[/|-](?P<Day>\d{1,2})[/|-](?P<Year>\d{2,4}))'))

,Original_txt,Date,Month,Day,Year
0,03/25/93 Total time of visit (in minutes):,03/25/93,03,25,93
1,6/18/85 Primary Care Doctor:,6/18/85,6,18,85
2,sshe plans to move as of 7/8/71 In-Home Servic...,7/8/71,7,8,71
3,7 on 9/27/75 Audit C Score Current:,9/27/75,9,27,75
4,2/6/96 sleep studyPain Treatment Pain Level (N...,2/6/96,2,6,96
...,...,...,...,...,...
495,1979 Family Psych History: Family History of S...,NaN,NaN,NaN,NaN
496,therapist and friend died in ~2006 Parental/Ca...,NaN,NaN,NaN,NaN
497,2008 partial thyroidectomy,NaN,NaN,NaN,NaN
498,sPt describes a history of sexual abuse as a c...,NaN,NaN,NaN,NaN


#### Putting a few of the pieces together
- Prelude to Ungraded task 2.2
    - A bit of data cleaning 
    - A bit of missing data handling
    - Doing so with RegEx 

In [45]:
# Let's go back to the CountriesOfTheWorld data.
# We previously loaded it in the CoTW_df DataFrame.
# We see lot's of missing data. 
# We had noticed previously that some of the numeric data is stored improperly (as string objects). 
CoTW_df.tail(6)

,Country,Region,Population,Area (sq. mi.),Pop. Density (per sq. mi.),Coastline (coast/area ratio),Net migration,Infant mortality (per 1000 births),GDP ($ per capita),Literacy (%),Phones (per 1000),Arable (%),Crops (%),Other (%),Climate,Birthrate,Deathrate,Agriculture,Industry,Service
221,Wallis and Futuna,OCEANIA,16025,274,"58,5","47,08",NaN,NaN,3700.0,"50,0","118,6",5,25,70,2,NaN,NaN,NaN,NaN,NaN
222,West Bank,NEAR EAST,2460492,5860,"419,9","0,00","2,98","19,62",800.0,NaN,"145,2","16,9","18,97","64,13",3,"31,67","3,92","0,09","0,28","0,63"
223,Western Sahara,NORTHERN AFRICA,273008,266000,"1,0","0,42",NaN,NaN,NaN,NaN,NaN,"0,02",0,"99,98",1,NaN,NaN,NaN,NaN,"0,4"
224,Yemen,NEAR EAST,21456188,527970,"40,6","0,36",0,"61,5",800.0,"50,2","37,2","2,78","0,24","96,98",1,"42,89","8,3","0,135","0,472","0,393"
225,Zambia,SUB-SAHARAN AFRICA,11502010,752614,"15,3","0,00",0,"88,29",800.0,"80,6","8,2","7,08","0,03","92,9",2,41,"19,93","0,22","0,29","0,489"
226,Zimbabwe,SUB-SAHARAN AFRICA,12236805,390580,"31,3","0,00",0,"67,69",1900.0,"90,7","26,8","8,32","0,34","91,34",2,"28,01","21,84","0,179","0,243","0,579"


In [46]:
# Let's check the dtypes to confirm this.
CoTW_df.dtypes

Country                                object
Region                                 object
Population                              int64
Area (sq. mi.)                          int64
Pop. Density (per sq. mi.)             object
Coastline (coast/area ratio)           object
Net migration                          object
Infant mortality (per 1000 births)     object
GDP ($ per capita)                    float64
Literacy (%)                           object
Phones (per 1000)                      object
Arable (%)                             object
Crops (%)                              object
Other (%)                              object
Climate                                object
Birthrate                              object
Deathrate                              object
Agriculture                            object
Industry                               object
Service                                object
dtype: object

In [47]:
# Let's convert the Literacy (%) entries into float() type. 
# We need to replace the ',' with '.'; following which, we can apply astype(float)
CoTW_df['Literacy (%)']=CoTW_df['Literacy (%)'].str.replace(',', '.').astype(float)
CoTW_df.tail(10)

,Country,Region,Population,Area (sq. mi.),Pop. Density (per sq. mi.),Coastline (coast/area ratio),Net migration,Infant mortality (per 1000 births),GDP ($ per capita),Literacy (%),Phones (per 1000),Arable (%),Crops (%),Other (%),Climate,Birthrate,Deathrate,Agriculture,Industry,Service
217,Vanuatu,OCEANIA,208869,12200,"17,1","20,72",0,"55,16",2900.0,53.0,"32,6","2,46","7,38","90,16",2,"22,72","7,82","0,26","0,12","0,62"
218,Venezuela,LATIN AMER. & CARIB,25730435,912050,"28,2","0,31","-0,04","22,2",4800.0,93.4,"140,1","2,95","0,92","96,13",2,"18,71","4,92","0,04","0,419","0,541"
219,Vietnam,ASIA (EX. NEAR EAST),84402966,329560,"256,1","1,05","-0,45","25,95",2500.0,90.3,"187,7","19,97","5,95","74,08",2,"16,86","6,22","0,209","0,41","0,381"
220,Virgin Islands,LATIN AMER. & CARIB,108605,1910,"56,9","9,84","-8,94","8,03",17200.0,NaN,"652,8","11,76","2,94","85,3",2,"13,96","6,43","0,01","0,19","0,8"
221,Wallis and Futuna,OCEANIA,16025,274,"58,5","47,08",NaN,NaN,3700.0,50.0,"118,6",5,25,70,2,NaN,NaN,NaN,NaN,NaN
222,West Bank,NEAR EAST,2460492,5860,"419,9","0,00","2,98","19,62",800.0,NaN,"145,2","16,9","18,97","64,13",3,"31,67","3,92","0,09","0,28","0,63"
223,Western Sahara,NORTHERN AFRICA,273008,266000,"1,0","0,42",NaN,NaN,NaN,NaN,NaN,"0,02",0,"99,98",1,NaN,NaN,NaN,NaN,"0,4"
224,Yemen,NEAR EAST,21456188,527970,"40,6","0,36",0,"61,5",800.0,50.2,"37,2","2,78","0,24","96,98",1,"42,89","8,3","0,135","0,472","0,393"
225,Zambia,SUB-SAHARAN AFRICA,11502010,752614,"15,3","0,00",0,"88,29",800.0,80.6,"8,2","7,08","0,03","92,9",2,41,"19,93","0,22","0,29","0,489"
226,Zimbabwe,SUB-SAHARAN AFRICA,12236805,390580,"31,3","0,00",0,"67,69",1900.0,90.7,"26,8","8,32","0,34","91,34",2,"28,01","21,84","0,179","0,243","0,579"


In [48]:
# reconfirm type change
CoTW_df.dtypes

Country                                object
Region                                 object
Population                              int64
Area (sq. mi.)                          int64
Pop. Density (per sq. mi.)             object
Coastline (coast/area ratio)           object
Net migration                          object
Infant mortality (per 1000 births)     object
GDP ($ per capita)                    float64
Literacy (%)                          float64
Phones (per 1000)                      object
Arable (%)                             object
Crops (%)                              object
Other (%)                              object
Climate                                object
Birthrate                              object
Deathrate                              object
Agriculture                            object
Industry                               object
Service                                object
dtype: object

#### Datetime object
    https://docs.python.org/3/library/datetime.html#strftime-strptime-behavior


In [49]:
date1='03-25-93'
date2="3/25/1993"
date3="25 Mar 1993"
print(date1==date3)

False


In [50]:
datetime_object1 = datetime.datetime.strptime(date1,"%m-%d-%y")
print(datetime_object1)

1993-03-25 00:00:00


In [51]:
datetime_object2 = datetime.datetime.strptime(date2,"%m/%d/%Y")
print(datetime_object2)

1993-03-25 00:00:00


In [52]:
datetime_object3 = datetime.datetime.strptime(date3,"%d %b %Y")
print(datetime_object3)
print(datetime_object1==datetime_object3)

1993-03-25 00:00:00
True


<b> Ungraded task 2.2: Clean the CoTW_df Dataframe</b>

Clean the data in the CoTW_df Dataframe as follows:

1. Create rule based methods (e.g., using regular expressions) to replace the column names as follows: 
    * Remove everything in brackets, e.g., replace "Coastline (coast/area ratio)" with "Coastline"
    * If there are multiple words in the title, then join them with underscores. As such, "Infant mortality (per 1000 births)" would become "Infant_mortality". 

2. Replace all the numeric values which have been stored as strings, into floats.

3. For all the missing numeric values (NaN), replace them with the Mean value available for the "Region". e.g., Wallis and Futuna is from OCEANIA region, so replace its missing Net_Migration information with the mean Net_Migration as determined for the values available for other countries in OCEANIA. 
    * Discuss alternatives for dealing with the NaN values. These alternatives may be dependent on type of column and/or the rows involved.

<b>Ungraded task 2.3: Find all the dates in the datetxt_df Series</b><br><br>

- Identify the (first instance of) date in each line of the Data Series.
    - If only month and year information is available, then assume that the day of the month is 1st.
    - If only year information is available, consider the month to be January.
- Create a DataFrame (as shown below) with two columns, the Position of Date in the original Series, and the Date identified, sorted in increasing order of the date. 

|     |   Position | Date       |
|----:|-----------:|:-----------|
|   0 |          9 | 10/4/1971  |
|   1 |         84 | 18/5/1971  |
|   2 |          2 | 8/7/1971   |
| ... |        ... |    ...     |
| ... |        ... |    ...     |
| 498 |        161 | 19/10/2016 |
| 499 |        413 | 1/11/2016  |


- Discuss: Assuming that you have access to a correct/ideal solution (e.g., the one provided), and you obtain some solution based on your processing, how will you quantitatively compare the quality of result you obtained w.r.to the ideal solution? If you consider several metrics, discuss pros and cons of such various metrics. Implement the corresponding metric and quantify the quality of result you obtain.
- A tentative solution has been provided in an accompanying file named sorted-dates.csv
    - Caveat: The solution provided may be faulty. If you find a better solution, please share with me. Please also let me know how you verified inaccuracies of the tentative solution.

<p style="font-size:134%;color:Deep Teal;">That's it folks!</p>

<img src="pics/tangram.png" alt="Bear in Tangram" width="500"/>